<a href="https://colab.research.google.com/github/thavyne-KDR/Detec-o_de_Ironia_em_Textos/blob/main/Irony-Classification-BERTimbau.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files

print("Por favor, selecione o arquivo 'classificacao-de-ironia.zip' para fazer o upload.")
uploaded = files.upload()

zip_file_name = list(uploaded.keys())[0]

print(f"\nArquivo '{zip_file_name}' enviado com sucesso!")

!unzip -q {zip_file_name}

print("\nArquivo descompactado! Os seguintes arquivos agora estão disponíveis:")

!ls

Por favor, selecione o arquivo 'classificacao-de-ironia.zip' para fazer o upload.


Saving classificacao-de-ironia.zip to classificacao-de-ironia.zip

Arquivo 'classificacao-de-ironia.zip' enviado com sucesso!

Arquivo descompactado! Os seguintes arquivos agora estão disponíveis:
classificacao-de-ironia.zip  sample_submission.csv  train.jsonl
sample_data		     test.csv


In [ ]:
from google.colab import files
import os

!pip install transformers torch scikit-learn -q

if not os.path.exists('train.jsonl'):
    print("Por favor, selecione o arquivo 'classificacao-de-ironia.zip' para fazer o upload.")
    uploaded = files.upload()
    zip_file_name = list(uploaded.keys())[0]
    !unzip -q {zip_file_name}
    print(f"\nArquivo '{zip_file_name}' descompactado com sucesso!")
else:
    print("Arquivos de dados já foram encontrados. Pulando o upload.")

print("\nArquivos no diretório atual:")
!ls

Arquivos de dados já foram encontrados. Pulando o upload.

Arquivos no diretório atual:
classificacao-de-ironia.zip  sample_submission.csv  train.jsonl
sample_data		     test.csv


In [ ]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification, get_linear_schedule_with_warmup
from torch.optim import AdamW
from sklearn.model_selection import StratifiedKFold
from tqdm.auto import tqdm
import gc

MODEL_NAME = 'neuralmind/bert-large-portuguese-cased'
N_SPLITS = 5
MAX_LEN = 256
BATCH_SIZE = 8
EPOCHS = 3
LEARNING_RATE = 1e-5
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

train_df = pd.read_json('train.jsonl', lines=True)
test_df = pd.read_csv('test.csv')

train_df['full_text'] = train_df['headline'].fillna('') + " " + train_df['text'].fillna('')
test_df['full_text'] = test_df['text'].fillna('')

class IronyDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts, self.labels, self.tokenizer, self.max_len = texts, labels, tokenizer, max_len
    def __len__(self): return len(self.texts)
    def __getitem__(self, item):
        text, label = str(self.texts[item]), self.labels[item]
        encoding = self.tokenizer.encode_plus(
            text, add_special_tokens=True, max_length=self.max_len,
            return_token_type_ids=False, padding='max_length',
            return_attention_mask=True, return_tensors='pt', truncation=True
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

class TestDataset(Dataset):
    def __init__(self, texts, tokenizer, max_len):
        self.texts, self.tokenizer, self.max_len = texts, tokenizer, max_len
    def __len__(self): return len(self.texts)
    def __getitem__(self, item):
        encoding = self.tokenizer.encode_plus(
            str(self.texts[item]), add_special_tokens=True, max_length=self.max_len,
            return_token_type_ids=False, padding='max_length',
            return_attention_mask=True, return_tensors='pt', truncation=True
        )
        return {'input_ids': encoding['input_ids'].flatten(), 'attention_mask': encoding['attention_mask'].flatten()}

def train_fn(data_loader, model, optimizer, device, scheduler):
    model.train()
    for d in tqdm(data_loader, desc="Training"):
        optimizer.zero_grad()
        outputs = model(
            input_ids=d["input_ids"].to(device),
            attention_mask=d["attention_mask"].to(device),
            labels=d["labels"].to(device)
        )
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        scheduler.step()

def eval_fn(data_loader, model, device):
    model.eval()
    final_predictions = []
    with torch.no_grad():
        for d in tqdm(data_loader, desc="Predicting"):
            outputs = model(
                input_ids=d["input_ids"].to(device),
                attention_mask=d["attention_mask"].to(device)
            )
            final_predictions.extend(outputs.logits.cpu().detach().numpy())
    return np.array(final_predictions)

print("Célula de configurações concluída. Tudo pronto para o treinamento.")

Célula de configurações concluída. Tudo pronto para o treinamento.


In [ ]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

test_dataset = TestDataset(test_df.full_text.values, tokenizer, MAX_LEN)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE * 2, shuffle=False, num_workers=2)

skf = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=42)

for fold, (train_idx, val_idx) in enumerate(skf.split(train_df, train_df['label'])):
    print(f"\n========== FOLD {fold+1}/{N_SPLITS} ==========")

    if os.path.exists(f'test_preds_fold_{fold}.npy'):
        print(f"Predições para o FOLD {fold+1} já existem. Pulando.")
        continue

    train_fold = train_df.iloc[train_idx]
    train_dataset = IronyDataset(train_fold.full_text.values, train_fold.label.values, tokenizer, MAX_LEN)
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)

    model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2).to(device)
    optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)
    scheduler = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps=0, num_training_steps=len(train_loader) * EPOCHS
    )

    for epoch in range(EPOCHS):
        print(f"  Epoch {epoch+1}/{EPOCHS}")
        train_fn(train_loader, model, optimizer, device, scheduler)

    test_preds = eval_fn(test_loader, model, device)
    np.save(f'test_preds_fold_{fold}.npy', test_preds)
    print(f"Predições do FOLD {fold+1} salvas com sucesso!")

    del model, train_loader, train_dataset
    gc.collect()
    torch.cuda.empty_cache()

print("\nTreinamento de todos os folds foi concluído!")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/155 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/648 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

...

Training:   0%|          | 0/1850 [00:00<?, ?it/s]

  Epoch 3/3


Training:   0%|          | 0/1850 [00:00<?, ?it/s]

Predicting:   0%|          | 0/19 [00:00<?, ?it/s]

Predições do FOLD 5 salvas com sucesso!

Treinamento de todos os folds foi concluído!


In [ ]:
all_test_preds = []

for fold in range(N_SPLITS):
    try:
        preds = np.load(f'test_preds_fold_{fold}.npy')
        all_test_preds.append(preds)
    except FileNotFoundError:
        print(f"ERRO: Arquivo 'test_preds_fold_{fold}.npy' não encontrado. Certifique-se de que o treinamento foi concluído.")
        break

if len(all_test_preds) == N_SPLITS:
    print(f"Carregadas {len(all_test_preds)} previsões de folds.")

    avg_preds = np.mean(all_test_preds, axis=0)

    final_preds = np.argmax(avg_preds, axis=1)

    submission_df = pd.DataFrame({'id': test_df['id'], 'label': final_preds})
    submission_df.to_csv('submission_final.csv', index=False)

    print("\nArquivo de submissão 'submission_final.csv' criado com sucesso!")
    files.download('submission_final.csv')

Carregadas 5 previsões de folds.

Arquivo de submissão 'submission_final.csv' criado com sucesso!


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>